In [ ]:
import math
import torchinfo
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import copy



from torch import nn
from System.pos_enc import *
from torch.optim.lr_scheduler import StepLR
from Data.MowingDetectionDataset_Parcelized import *
from System.pos_enc import *
from System.Engine import *
from System.Engine_Tr import *
from helpers.various import *
from torchmetrics import Accuracy, ConfusionMatrix, F1Score, Recall, Precision
from torchinfo import summary


transforms = A.Compose([
        #A.Normalize(mean=[-0.3357, -0.3259],#####how many channels to include?
                   # std=[0.4688, 0.4685]),
        ToTensorV2()
    ])




###padding is used because there is the possibility to have nan at the ts start###
###also the argument nan_free is passed to interpolate the ts###
train_dt = ParcelMowingDataset(root_path = root_dataset_path,labels_path = path_train, trsm = transforms,max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
train_dloader = DataLoader(train_dt, sampler=ImbalancedDatasetSampler(train_dt), 
                           batch_size=train_batch, shuffle=False, num_workers=0)

val_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_val, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
val_dloader = DataLoader(val_dt, batch_size=val_batch, shuffle=False, num_workers=0)

test_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_test, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
test_dloader = DataLoader(test_dt, batch_size=test_batch, shuffle=False, num_workers=0)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner

###callbacks to use###
callbacks = [
    EarlyStopping(monitor="accuracy_macro/val", mode="max", patience=60),
    ModelCheckpoint(monitor="accuracy_macro/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]

###select model to use### recomended is MLP
model = Model_MLP_MV()
###trainer###
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    max_epochs=number_of_epochs,
    callbacks=callbacks,
    default_root_dir=save_dir,
    enable_progress_bar = False,
)

###start train and evaluation###
trainer.fit(model, train_dataloaders=train_dloader, val_dataloaders=val_dloader)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner

###callbacks to use###
callbacks = [
    EarlyStopping(monitor="accuracy_w/val", mode="max", patience=60),
    ModelCheckpoint(monitor="accuracy_w/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]

###select model to use###
model = ECGTRMODEL()
###trainer###
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    max_epochs=number_of_epochs,
    callbacks=callbacks,
    default_root_dir=save_dir,
    enable_progress_bar = False,
    #resume_from_checkpoint= './master_thesis/lightning_logs/version_212/checkpoints/epoch=22-step=1679.ckpt'
)

###start train and evaluation###
trainer.fit(model, train_dataloaders=train_dloader, val_dataloaders=val_dloader)

In [ ]:
###load the best model###
best_model = Model.load_from_checkpoint('./checkpoints/pretrained_models/lightning_logs/version_3/checkpoints/epoch=74-step=5475.ckpt')
###test the model in the test dataset###
trainer = pl.Trainer(accelerator="gpu", devices=1)
trainer.test(best_model, dataloaders=test_dloader)

In [ ]:
###display the confusion matrix for the testing part of the best model###
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt


class_labels = ['0 times', '1 times','2 times', '3 times', '4 times']
cm = best_model.test_confusion_matrix.compute().cpu().numpy()

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_labels )
plt.figure(figsize=(5,5), dpi=100)
ax = plt.axes()

disp.plot(ax=ax)
plt.xticks(rotation=90)
plt.show()

In [ ]:
dt, lbl = next(iter(test_dloader))
print(dt.shape)
#print(lbl)
with torch.no_grad():
    y = best_model(dt)
print('Pred',torch.argmax(y,dim=1), 'GT',lbl)